## Importing libraries 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import pickle
import json
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_rows', None)

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# CARGO DATOS JSON

In [3]:
#with open('data.json',encoding='utf-8') as file:
with open('data.json',encoding='latin') as file:
    payload = json.load(file)

# Imprimo data y customer

In [4]:
customer = payload['customer']
print(customer)
input_features = pd.DataFrame(payload['data'],
                                      index=[i for i in range(len(payload['data']))])
input_features

banesco_panama


,APPLICANT_ID,Cargo,DescripcionIndustria,Descripcion_Nivel_Educacion,Estadocivil,Genero,IngresoMensual,SALDO_PAS_UM,TiempoRelacionBanco,antiguedad_empleo,cantidad_Dependientes,ciudad,edad,max_dias_atraso,monto_cuotas,pais,prom_saldo_actual,provincia,score
0,test006,OTROS,ALIMENTOS,OTROS,SINGLE,MALE,3200.0000,nan,3,3.0466,0,PANAMA,20.3425,nan,nan,PANAMA,nan,PANAMAOESTE,nan
1,test005,NaN,MERCANCIA,Universitario,SINGLE,MALE,108.3400,121345.7700,22,1.6685,0,PANAMA,52.3726,0.0000,0.0000,PANAMA,414.6567,PANAMA,692.0000


____

## Lectura de artefactos

In [5]:
with open('../artefactos/categorical_features.pkl', 'rb') as f:
    categorical_columns = pickle.load(f)

In [6]:
categorical_columns   

['Descripcion_Nivel_Educacion',
 'Estadocivil',
 'Genero',
 'Cargo',
 'DescripcionIndustria',
 'pais',
 'provincia',
 'ciudad']

In [7]:
with open('../artefactos/feature_headers.pkl', 'rb') as f:
    feature_headers = pickle.load(f)

In [8]:
feature_headers  

Index(['cantidad_Dependientes', 'TiempoRelacionBanco', 'IngresoMensual',
       'edad', 'antiguedad_empleo', 'score', 'max_dias_atraso',
       'ratio_i_sprom', 'ratio_i_spas', 'ratio_i_cuotas',
       'Descripcion_Nivel_Educacion__Maestria',
       'Descripcion_Nivel_Educacion__OTROS',
       'Descripcion_Nivel_Educacion__Secundaria',
       'Descripcion_Nivel_Educacion__SinEscolaridad',
       'Descripcion_Nivel_Educacion__Universitario', 'Estadocivil__MARRIED',
       'Estadocivil__OTHER', 'Estadocivil__SINGLE', 'Genero__FEMALE',
       'Genero__MALE', 'Cargo__ABOGADO', 'Cargo__ADMINISTRADOR',
       'Cargo__AGENTECOMERCIAL', 'Cargo__COMERCIANTE', 'Cargo__DIRECTOR',
       'Cargo__GERENTESUBGERENTE', 'Cargo__OTROS',
       'DescripcionIndustria__ACUICULTURAPESCADERIA',
       'DescripcionIndustria__ADMINISTRACION',
       'DescripcionIndustria__AGRICULTURAAGRONOMIA',
       'DescripcionIndustria__ALIMENTOS',
       'DescripcionIndustria__ARQUITECTURAYURBANISMO',
       'DescripcionI

In [9]:
with open('../artefactos/xgb.pkl', 'rb') as f:
    model = pickle.load(f)

In [10]:
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample=1.0,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=2.4622198171652845, learning_rate=0.09175399398624959,
              max_delta_step=0, max_depth=4, min_child_weight=1, missing=nan,
              n_estimators=499, n_jobs=1, nthread=None,
              objective='binary:logistic', random_state=1, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1.0, verbosity=1)

In [11]:
with open('../artefactos/threshold.pkl', 'rb') as f:
    threshold = pickle.load(f)

In [12]:
threshold

0.66

____

## Pipelines

In [13]:
from src.pipelines.banesco_panama.banesco_panama_pre_processing_variables import BasicVariableTransformer_banesco_panama
from sklearn.pipeline import Pipeline

In [14]:
#input_features['cantidad_Dependientes'] = 40
#input_features['edad'] = 300
#input_features['Estadocivil'] = 'N'
#input_features['IngresoMensual'] = 0

In [15]:
input_features.T

,0,1
APPLICANT_ID,test006,test005
Cargo,OTROS,NaN
DescripcionIndustria,ALIMENTOS,MERCANCIA
Descripcion_Nivel_Educacion,OTROS,Universitario
Estadocivil,SINGLE,SINGLE
Genero,MALE,MALE
IngresoMensual,3200.0000,108.3400
SALDO_PAS_UM,NaN,121345.7700
TiempoRelacionBanco,3,22
antiguedad_empleo,3.0466,1.6685


In [16]:
 '''      if customer == 'banesco':
            
                pipeline_banesco = Pipeline([
                                ('cantidad_Dependientes', BasicVariableTransformer('cantidad_Dependientes')),
                                ('edad', BasicVariableTransformer('edad')),
                                ('Estadocivil', BasicVariableTransformer('Estadocivil')),
                                ('Genero', BasicVariableTransformer('Genero')),
                                ('provincia', BasicVariableTransformer('provincia')),
                                ('IngresoMensual', BasicVariableTransformer('IngresoMensual')),


                                ('score', BasicVariableTransformer('score')),
                                ('TiempoRelacionBanco', BasicVariableTransformer('TiempoRelacionBanco')),
                                ('max_dias_atraso', BasicVariableTransformer('max_dias_atraso')),
                                ('antiguedad_empleo', BasicVariableTransformer('antiguedad_empleo')),
                                ('Cargo', BasicVariableTransformer('Cargo')),
                                ('DescripcionIndustria', BasicVariableTransformer('DescripcionIndustria')),


                                ('Descripcion_Nivel_Educacion', BasicVariableTransformer('Descripcion_Nivel_Educacion')),
                                ('ciudad', BasicVariableTransformer('ciudad')),
                                ('pais', BasicVariableTransformer('pais'))

                            ])
            
                input_features = pipeline_banesco.transform(input_features)
                
                input_features['IngresoMensual'] = input_features['IngresoMensual'].replace(0, np.nan)
                input_features['ratio_i_sprom'] = input_features['prom_saldo_actual']/(input_features['IngresoMensual'])
                input_features['ratio_i_spas'] = input_features['SALDO_PAS_UM']/(input_features['IngresoMensual'])
                input_features['ratio_i_cuotas'] = input_features['monto_cuotas']/(input_features['IngresoMensual'])
'''

"      if customer == 'banesco':\n           \n               pipeline_banesco = Pipeline([\n                               ('cantidad_Dependientes', BasicVariableTransformer('cantidad_Dependientes')),\n                               ('edad', BasicVariableTransformer('edad')),\n                               ('Estadocivil', BasicVariableTransformer('Estadocivil')),\n                               ('Genero', BasicVariableTransformer('Genero')),\n                               ('provincia', BasicVariableTransformer('provincia')),\n                               ('IngresoMensual', BasicVariableTransformer('IngresoMensual')),\n\n\n                               ('score', BasicVariableTransformer('score')),\n                               ('TiempoRelacionBanco', BasicVariableTransformer('TiempoRelacionBanco')),\n                               ('max_dias_atraso', BasicVariableTransformer('max_dias_atraso')),\n                               ('antiguedad_empleo', BasicVariableTransformer('anti

In [17]:
input_features['ciudad'] = input_features['ciudad'].str.upper()

In [18]:
input_features['IngresoMensual'] = input_features['IngresoMensual'].replace(0, np.nan)
input_features['ratio_i_sprom'] = input_features['prom_saldo_actual']/(input_features['IngresoMensual'])
input_features['ratio_i_spas'] = input_features['SALDO_PAS_UM']/(input_features['IngresoMensual'])
input_features['ratio_i_cuotas'] = input_features['monto_cuotas']/(input_features['IngresoMensual'])

In [22]:
pipeline_banesco_panama = Pipeline([

    
                ('cantidad_Dependientes', BasicVariableTransformer_banesco_panama('cantidad_Dependientes')),
                ('edad', BasicVariableTransformer_banesco_panama('edad')),
                ('Estadocivil', BasicVariableTransformer_banesco_panama('Estadocivil')),
                ('Genero', BasicVariableTransformer_banesco_panama('Genero')),
                ('provincia', BasicVariableTransformer_banesco_panama('provincia')),
                #('IngresoMensual', BasicVariableTransformer_banesco_panama('IngresoMensual')),
                
    
                ('score', BasicVariableTransformer_banesco_panama('score')),
                ('TiempoRelacionBanco', BasicVariableTransformer_banesco_panama('TiempoRelacionBanco')),
                ('max_dias_atraso', BasicVariableTransformer_banesco_panama('max_dias_atraso')),
                ('antiguedad_empleo', BasicVariableTransformer_banesco_panama('antiguedad_empleo')),
                ('Cargo', BasicVariableTransformer_banesco_panama('Cargo')),
                ('DescripcionIndustria', BasicVariableTransformer_banesco_panama('DescripcionIndustria')),
    
    
                ('Descripcion_Nivel_Educacion', BasicVariableTransformer_banesco_panama('Descripcion_Nivel_Educacion')),
                ('ciudad', BasicVariableTransformer_banesco_panama('ciudad')),
                ('pais', BasicVariableTransformer_banesco_panama('pais')),
    
               # ('ratio_i_sprom', BasicVariableTransformer('ratio_i_sprom')),
               # ('ratio_i_spas', BasicVariableTransformer('ratio_i_spas')),
               # ('ratio_i_cuotas', BasicVariableTransformer('ratio_i_cuotas'))
    
            ])
            

In [23]:
%%time
input_features = pipeline_banesco_panama.transform(input_features)

['0', '6']
1.0
The Data sent from customer has cantidad_Dependientes with value: 0
The element cantidad_Dependientes with value 0 now is 0.0
['18', '76']
54.948
The Data sent from customer has edad with value: 20.342465753424662
The element edad with value 20.342465753424662 now is 20.342465753424662
['0', '57']
57.37
The Data sent from customer has edad with value: 20.342465753424662
The element edad with value 20.342465753424662 now is 20.342465753424662
['MARRIED', 'OTHER', 'SINGLE']
SINGLE
The Data sent from customer has Estadocivil with value: SINGLE
The element Estadocivil with value SINGLE now is SINGLE
['FEMALE', 'MALE']
MALE
The Data sent from customer has Genero with value: MALE
The element Genero with value MALE now is MALE
['CHIRIQUI', 'COCLE', 'COLON', 'HERRERA', 'OTROS', 'PANAMA', 'PANAMAOESTE', 'VERAGUAS']
OTROS
The Data sent from customer has provincia with value: PANAMAOESTE
The element provincia with value PANAMAOESTE now is PANAMAOESTE
['0', '874']
510.0
The Data sen

In [24]:
input_features.T

,0,1
APPLICANT_ID,test006,test005
Cargo,OTROS,ABOGADO
DescripcionIndustria,ALIMENTOS,MERCANCIA
Descripcion_Nivel_Educacion,OTROS,OTROS
Estadocivil,SINGLE,SINGLE
Genero,MALE,MALE
IngresoMensual,3200.0000,108.3400
SALDO_PAS_UM,NaN,121345.7700
TiempoRelacionBanco,3.0000,22.0000
antiguedad_empleo,3.0466,1.6685


In [25]:
input_features = pd.get_dummies(input_features, prefix_sep="__", columns=categorical_columns)

In [26]:
input_features = input_features.reindex(columns=feature_headers, fill_value=0)
#input_features.fillna(0,inplace=True)

____

## Variable importance

In [27]:
d1 = pd.DataFrame(model.feature_importances_)
d2 = pd.DataFrame(feature_headers)
d = pd.concat([d2, d1], axis=1)
d.columns = ["variable", "weight"]
d = d.sort_values('weight',ascending=False)
d

,variable,weight
5,score,0.0844
8,ratio_i_spas,0.0633
35,DescripcionIndustria__CONTABILIDAD,0.0365
27,DescripcionIndustria__ACUICULTURAPESCADERIA,0.0355
6,max_dias_atraso,0.0346
20,Cargo__ABOGADO,0.0339
26,Cargo__OTROS,0.0336
30,DescripcionIndustria__ALIMENTOS,0.0329
4,antiguedad_empleo,0.0326
3,edad,0.0305


____

## Model testing

In [28]:
scores = model.predict_proba(input_features)
scores

array([[0.3589915, 0.6410085],
       [0.0283643, 0.9716357]], dtype=float32)

In [29]:
labels = [1 if sc[1] > threshold else 0 for sc in scores]
labels

[0, 1]

In [30]:
responses_labels  = {0 : "RECHAZADO", 1 : "APROBADO"}

___

## API output

In [31]:
predictions = [{'approval_score':row[0][1], 
                'label':responses_labels[row[1]]} for row in zip(scores, labels)]        
predictions

[{'approval_score': 0.6410085, 'label': 'RECHAZADO'},
 {'approval_score': 0.9716357, 'label': 'APROBADO'}]